In [1]:
""" Implementation of logistic ordinal regression (aka proportional odds) model

author: fernando pedorosa, date: march-2013
"""

# Import standard packages
import numpy as np

# additional packages
from sklearn import metrics
from scipy import linalg, optimize, sparse
import warnings

BIG = 1e10
SMALL = 1e-12


def phi(t):
    ''' logistic function, returns 1 / (1 + exp(-t)) '''
    
    idx = t > 0
    out = np.empty(t.size, dtype=np.float)
    out[idx] = 1. / (1 + np.exp(-t[idx]))
    exp_t = np.exp(t[~idx])
    out[~idx] = exp_t / (1. + exp_t)
    return out

def log_logistic(t):
    ''' (minus) logistic loss function, returns log(1 / (1 + exp(-t))) '''
    
    idx = t > 0
    out = np.zeros_like(t)
    out[idx] = np.log(1 + np.exp(-t[idx]))
    out[~idx] = (-t[~idx] + np.log(1 + np.exp(t[~idx])))
    return out


def ordinal_logistic_fit(X, y, alpha=0, l1_ratio=0, n_class=None, max_iter=10000,
                         verbose=False, solver='TNC', w0=None):
    '''
    Ordinal logistic regression or proportional odds model.
    Uses scipy's optimize.fmin_slsqp solver.

    Parameters
    ----------
    X : {array, sparse matrix}, shape (n_samples, n_feaures)
        Input data
    y : array-like
        Target values
    max_iter : int
        Maximum number of iterations
    verbose: bool
        Print convergence information

    Returns
    -------
    w : array, shape (n_features,)
        coefficients of the linear model
    theta : array, shape (k,), where k is the different values of y
        vector of thresholds
    '''

    X = np.asarray(X)
    y = np.asarray(y)
    w0 = None

    if not X.shape[0] == y.shape[0]:
        raise ValueError('Wrong shape for X and y')

    # .. order input ..
    idx = np.argsort(y)
    idx_inv = np.zeros_like(idx)
    idx_inv[idx] = np.arange(idx.size)
    X = X[idx]
    y = y[idx].astype(np.int)
    # make them continuous and start at zero
    unique_y = np.unique(y)
    for i, u in enumerate(unique_y):
        y[y == u] = i
    unique_y = np.unique(y)

    # .. utility arrays used in f_grad ..
    alpha = 0.
    k1 = np.sum(y == unique_y[0])
    E0 = (y[:, np.newaxis] == np.unique(y)).astype(np.int)
    E1 = np.roll(E0, -1, axis=-1)
    E1[:, -1] = 0.
    E0, E1 = map(sparse.csr_matrix, (E0.T, E1.T))

    def f_obj(x0, X, y):
        """
        Objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        if np.any(c > 0):
            return BIG

        #loss = -(c[idx] + np.log(np.exp(-c[idx]) - 1)).sum()
        loss = -np.log(1 - np.exp(c)).sum()

        loss += b.sum() + log_logistic(b).sum() \
            + log_logistic(a).sum() \
            + .5 * alpha * w.dot(w) - np.log(z).sum()  # penalty
        if np.isnan(loss):
            pass
            #import ipdb; ipdb.set_trace()
        return loss

    def f_grad(x0, X, y):
        """
        Gradient of the objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        # gradient for w
        phi_a = phi(a)
        phi_b = phi(b)
        grad_w = -X[k1:].T.dot(phi_b) + X.T.dot(1 - phi_a) + alpha * w

        # gradient for theta
        idx = c > 0
        tmp = np.empty_like(c)
        tmp[idx] = 1. / (np.exp(-c[idx]) - 1)
        tmp[~idx] = np.exp(c[~idx]) / (1 - np.exp(c[~idx])) # should not need
        grad_theta = (E1 - E0)[:, k1:].dot(tmp) \
            + E0[:, k1:].dot(phi_b) - E0.dot(1 - phi_a)

        grad_theta[:-1] += 1. / np.diff(theta_0)
        grad_theta[1:] -= 1. / np.diff(theta_0)
        out = np.concatenate((grad_w, grad_theta))
        return out

    def f_hess(x0, s, X, y):
        x0 = np.asarray(x0)
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        D = np.diag(phi(a) * (1 - phi(a)))
        D_= np.diag(phi(b) * (1 - phi(b)))
        D1 = np.diag(np.exp(-c) / (np.exp(-c) - 1) ** 2)
        Ex = (E1 - E0)[:, k1:].toarray()
        Ex0 = E0.toarray()
        H_A = X[k1:].T.dot(D_).dot(X[k1:]) + X.T.dot(D).dot(X)
        H_C = - X[k1:].T.dot(D_).dot(E0[:, k1:].T.toarray()) \
            - X.T.dot(D).dot(E0.T.toarray())
        H_B = Ex.dot(D1).dot(Ex.T) + Ex0[:, k1:].dot(D_).dot(Ex0[:, k1:].T) \
            - Ex0.dot(D).dot(Ex0.T)

        p_w = H_A.shape[0]
        tmp0 = H_A.dot(s[:p_w]) + H_C.dot(s[p_w:])
        tmp1 = H_C.T.dot(s[:p_w]) + H_B.dot(s[p_w:])
        return np.concatenate((tmp0, tmp1))

        import ipdb; ipdb.set_trace()
        import pylab as pl
        pl.matshow(H_B)
        pl.colorbar()
        pl.title('True')
        import numdifftools as nd
        Hess = nd.Hessian(lambda x: f_obj(x, X, y))
        H = Hess(x0)
        pl.matshow(H[H_A.shape[0]:, H_A.shape[0]:])
        #pl.matshow()
        pl.title('estimated')
        pl.colorbar()
        pl.show()


    def grad_hess(x0, X, y):
        grad = f_grad(x0, X, y)
        hess = lambda x: f_hess(x0, x, X, y)
        return grad, hess

    x0 = np.random.randn(X.shape[1] + unique_y.size) / X.shape[1]
    if w0 is not None:
        x0[:X.shape[1]] = w0
    else:
        x0[:X.shape[1]] = 0.
    x0[X.shape[1]:] = np.sort(unique_y.size * np.random.rand(unique_y.size))

    #print('Check grad: %s' % optimize.check_grad(f_obj, f_grad, x0, X, y))
    #print(optimize.approx_fprime(x0, f_obj, 1e-6, X, y))
    #print(f_grad(x0, X, y))
    #print(optimize.approx_fprime(x0, f_obj, 1e-6, X, y) - f_grad(x0, X, y))
    #import ipdb; ipdb.set_trace()

    def callback(x0):
        x0 = np.asarray(x0)
        # print('Check grad: %s' % optimize.check_grad(f_obj, f_grad, x0, X, y))
        if verbose:
        # check that gradient is correctly computed
            print('OBJ: %s' % f_obj(x0, X, y))

    if solver == 'TRON':
        import pytron
        out = pytron.minimize(f_obj, grad_hess, x0, args=(X, y))
    else:
        options = {'maxiter' : max_iter, 'disp': 0, 'maxfun':10000}
        out = optimize.minimize(f_obj, x0, args=(X, y), method=solver,
            jac=f_grad, hessp=f_hess, options=options, callback=callback)

    if not out.success:
        warnings.warn(out.message)
    w, theta = np.split(out.x, [X.shape[1]])
    return w, theta


def ordinal_logistic_predict(w, theta, X):
    """
    Parameters
    ----------
    w : coefficients obtained by ordinal_logistic
    theta : thresholds
    """
    unique_theta = np.sort(np.unique(theta))
    out = X.dot(w)
    unique_theta[-1] = np.inf # p(y <= max_level) = 1
    tmp = out[:, None].repeat(unique_theta.size, axis=1)
    return np.argmax(tmp < unique_theta, axis=1)

def main():
    DOC = """
================================================================================
    Compare the prediction accuracy of different models on the boston dataset
================================================================================
    """
    print(DOC)
    from sklearn import cross_validation, datasets
    boston = datasets.load_boston()
    X, y = boston.data, np.round(boston.target)
    #X -= X.mean()
    y -= y.min()

    idx = np.argsort(y)
    X = X[idx]
    y = y[idx]
    cv = cross_validation.ShuffleSplit(y.size, n_iter=50, test_size=.1, random_state=0)
    score_logistic = []
    score_ordinal_logistic = []
    score_ridge = []
    for i, (train, test) in enumerate(cv):
        #test = train
        if not np.all(np.unique(y[train]) == np.unique(y)):
            # we need the train set to have all different classes
            continue
        assert np.all(np.unique(y[train]) == np.unique(y))
        train = np.sort(train)
        test = np.sort(test)
        w, theta = ordinal_logistic_fit(X[train], y[train], verbose=True,
                                        solver='TNC')
        pred = ordinal_logistic_predict(w, theta, X[test])
        s = metrics.mean_absolute_error(y[test], pred)
        print('ERROR (ORDINAL)  fold %s: %s' % (i+1, s))
        score_ordinal_logistic.append(s)

        from sklearn import linear_model
        clf = linear_model.LogisticRegression(C=1.)
        clf.fit(X[train], y[train])
        pred = clf.predict(X[test])
        s = metrics.mean_absolute_error(y[test], pred)
        print('ERROR (LOGISTIC) fold %s: %s' % (i+1, s))
        score_logistic.append(s)

        from sklearn import linear_model
        clf = linear_model.Ridge(alpha=1.)
        clf.fit(X[train], y[train])
        pred = np.round(clf.predict(X[test]))
        s = metrics.mean_absolute_error(y[test], pred)
        print('ERROR (RIDGE)    fold %s: %s' % (i+1, s))
        score_ridge.append(s)


    print()
    print('MEAN ABSOLUTE ERROR (ORDINAL LOGISTIC):    %s' % np.mean(score_ordinal_logistic))
    print('MEAN ABSOLUTE ERROR (LOGISTIC REGRESSION): %s' % np.mean(score_logistic))
    print('MEAN ABSOLUTE ERROR (RIDGE REGRESSION):    %s' % np.mean(score_ridge))
    # print('Chance level is at %s' % (1. / np.unique(y).size))
    
    return np.mean(score_ridge)
    
if __name__ == '__main__':
    out = main()    
    print(out)



    Compare the prediction accuracy of different models on the boston dataset
    


C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:264: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:389: RuntimeWarning: Method TNC does not use Hessian-vector product information (hessp).
  'information (hessp).' % method, RuntimeWarning)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:221: OptimizeWarning: Unknown solver options: maxfun


OBJ: 4012.91711645
OBJ: 3439.25873597
OBJ: 3103.25984713
OBJ: 2844.50276968
OBJ: 2741.425208
OBJ: 2670.58896237
OBJ: 2523.65377924
OBJ: 2465.25321517
OBJ: 2344.45453181
OBJ: 2219.57954445
OBJ: 2214.51606859
OBJ: 2120.93149752
OBJ: 2056.19240648
OBJ: 2018.33691986
OBJ: 1936.2943995
OBJ: 1923.11099152
OBJ: 1909.75933152
OBJ: 1827.36993635
OBJ: 1703.04131412
OBJ: 1672.14350011
OBJ: 1584.72843229
OBJ: 1553.75406106
OBJ: 1506.51506002
OBJ: 1488.18752491
OBJ: 1475.94375678
OBJ: 1455.07273913
OBJ: 1451.00710937
OBJ: 1442.26449478
OBJ: 1376.82320441
OBJ: 1369.90706724
OBJ: 1361.23302539
OBJ: 1359.46736456
OBJ: 1359.38873148
OBJ: 1357.7611597
OBJ: 1357.23914345
OBJ: 1353.44007004
OBJ: 1350.17131436
OBJ: 1342.25540912
OBJ: 1331.59873287
OBJ: 1330.91844521
OBJ: 1330.23654786
OBJ: 1330.14882952
OBJ: 1330.11699612
OBJ: 1330.11628346
OBJ: 1330.11578469
OBJ: 1330.11520761
OBJ: 1330.11428798
OBJ: 1330.11398001
OBJ: 1330.11385383
OBJ: 1330.11376102
OBJ: 1330.11372485
OBJ: 1330.11371824
OBJ: 1330.113713

OBJ: 1883.7048978
OBJ: 1845.26390183
OBJ: 1522.87576637
OBJ: 1492.03340741
OBJ: 1444.77711637
OBJ: 1420.16741082
OBJ: 1403.25205219
OBJ: 1401.82951736
OBJ: 1399.0670043
OBJ: 1396.43533141
OBJ: 1395.18742431
OBJ: 1394.78836643
OBJ: 1393.72325196
OBJ: 1390.80332897
OBJ: 1369.00491007
OBJ: 1368.02120227
OBJ: 1352.85636505
OBJ: 1330.49476291
OBJ: 1324.73226607
OBJ: 1323.88461291
OBJ: 1323.70632768
OBJ: 1323.65621671
OBJ: 1323.64913539
OBJ: 1323.64844719
OBJ: 1323.64820488
OBJ: 1323.64637368
OBJ: 1323.64177282
OBJ: 1323.63878667
OBJ: 1323.6381336
OBJ: 1323.63754218
OBJ: 1323.63382679
OBJ: 1323.63190779
OBJ: 1323.63160738
OBJ: 1323.6315829
OBJ: 1323.63157889
OBJ: 1323.63157566
OBJ: 1323.63157417
OBJ: 1323.63157176
OBJ: 1323.63157161
OBJ: 1323.63157145
OBJ: 1323.63157143
OBJ: 1323.63157143
ERROR (ORDINAL)  fold 13: 3.70588235294
ERROR (LOGISTIC) fold 13: 4.60784313725
ERROR (RIDGE)    fold 13: 4.43137254902
OBJ: 3894.16202236
OBJ: 3599.6642306
OBJ: 3218.92313411
OBJ: 3107.59957204
OBJ: 2669.9

OBJ: 1317.25132973
OBJ: 1317.23678251
OBJ: 1317.22934827
OBJ: 1317.22365272
OBJ: 1317.21987616
OBJ: 1317.21968626
OBJ: 1317.21928024
OBJ: 1317.21665152
OBJ: 1317.21641309
OBJ: 1317.21545767
OBJ: 1317.21476006
OBJ: 1317.21432468
OBJ: 1317.21000764
OBJ: 1317.20886259
OBJ: 1317.20027124
OBJ: 1317.1986253
OBJ: 1317.19825964
OBJ: 1317.1981857
OBJ: 1317.1981762
OBJ: 1317.19817574
OBJ: 1317.19817552
OBJ: 1317.198175
OBJ: 1317.19817452
OBJ: 1317.19817448
OBJ: 1317.19817447
ERROR (ORDINAL)  fold 33: 3.09803921569
ERROR (LOGISTIC) fold 33: 4.05882352941
ERROR (RIDGE)    fold 33: 3.62745098039
OBJ: 4259.43650014
OBJ: 3310.50474374
OBJ: 3190.9661024
OBJ: 3040.8544989
OBJ: 2946.22598116
OBJ: 2486.65464935
OBJ: 2349.87775777
OBJ: 2218.34757947
OBJ: 2164.97858189
OBJ: 2058.33252663
OBJ: 1957.07443027
OBJ: 1885.09914643
OBJ: 1842.2357818
OBJ: 1788.26425967
OBJ: 1723.55818726
OBJ: 1708.10948634
OBJ: 1646.09233408
OBJ: 1573.94023279
OBJ: 1555.13953528
OBJ: 1475.45227362
OBJ: 1452.10381996
OBJ: 1420.2624

OBJ: 1319.3373579
OBJ: 1317.25868439
OBJ: 1316.54105799
OBJ: 1316.52948253
OBJ: 1316.40190052
OBJ: 1316.33358587
OBJ: 1316.28969967
OBJ: 1316.26777277
OBJ: 1316.26203717
OBJ: 1316.26040706
OBJ: 1316.25965095
OBJ: 1316.25918534
OBJ: 1316.25642847
OBJ: 1316.25265531
OBJ: 1316.25100294
OBJ: 1316.25076259
OBJ: 1316.25067044
OBJ: 1316.25066059
OBJ: 1316.25065872
OBJ: 1316.25064448
OBJ: 1316.25058701
OBJ: 1316.2505674
OBJ: 1316.25056194
OBJ: 1316.25056106
OBJ: 1316.25056091
OBJ: 1316.25056085
OBJ: 1316.25056083
OBJ: 1316.25056083
ERROR (ORDINAL)  fold 42: 3.01960784314
ERROR (LOGISTIC) fold 42: 3.52941176471
ERROR (RIDGE)    fold 42: 4.13725490196
OBJ: 4134.53658462
OBJ: 3342.58482668
OBJ: 3020.58143051
OBJ: 2567.95336582
OBJ: 2293.72591692
OBJ: 2245.03510788
OBJ: 2138.23665088
OBJ: 2093.36542273
OBJ: 2050.38953721
OBJ: 2028.69606413
OBJ: 1933.57557328
OBJ: 1875.60143682
OBJ: 1858.74832385
OBJ: 1779.74258765
OBJ: 1718.11210881
OBJ: 1702.03842729
OBJ: 1610.47241579
OBJ: 1580.85315418
OBJ: 154